In [61]:
import pandas as pd
import matplotlib as plt
import re
import numpy as np

In [62]:
df = pd.read_csv("./mercadoimobiliario.csv", sep= ';')
# extração e correção variavel valor
df['Valor'] = df['Valor'].str.replace(',', ".")
df['Valor'].str.strip()
df['Valor'] = pd.to_numeric(df['Valor'])

df['Data'] = pd.to_datetime(df['Data'], dayfirst=True, format='%d/%m/%Y')
print(df.columns)

df.columns = df.columns.str.strip()
df['tipo_transacao'] = df['Info'].apply(lambda x: x.split('_')[0])
df['tipo_credito'] = df.apply(lambda row: row['Info'].split('_')[1] if row['tipo_transacao'] == 'credito' else None, axis=1)

df['estado'] = df['Info'].apply(lambda x: x.split('_')[-1])
df_organizado_data = df.sort_values(by='Data')
df_organizado_data.columns = ['data', 'info', 'valor', 'tipo_transacao', 'tipo_credito', 'estado']
df_organizado_data = df_organizado_data.reset_index(drop=True)

print(df['tipo_transacao'].unique())
print(df['tipo_credito'].unique())


Index(['Data', 'Info', 'Valor'], dtype='object')
['credito' 'imoveis' 'direcionamento' 'fontes' 'indices' 'contabil']
['estoque' 'contratacao' None]


In [63]:
filtro_imoveis = df_organizado_data[df_organizado_data['tipo_transacao'].str.contains(r'\bimoveis\b', regex = True, case = False)]
filtro_dormitorios_3 = filtro_imoveis[filtro_imoveis['info'].str.contains(r'\wdormitorio\w', regex = True, case = False)]
filtro_dormitorios_3 = filtro_dormitorios_3[~filtro_dormitorios_3['info'].str.contains(r'\wmais\w', regex=True, case= False)]
filtro_dormitorios_3['dormitorios'] = filtro_dormitorios_3['info'].str.extract(r'dormitorio_(\d+)')
filtro_dormitorios_4 = filtro_imoveis[filtro_imoveis['info'].str.contains(r'\wmais\w', regex = True, case = False)]
filtro_dormitorios_4['dormitorios'] = filtro_dormitorios_4['info'].str.extract(r'dormitorio_(\d+)')

C:\Users\diogo\AppData\Local\Temp\ipykernel_17824\2504305013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_dormitorios_4['dormitorios'] = filtro_dormitorios_4['info'].str.extract(r'dormitorio_(\d+)')


In [64]:
imoveis_dormitorios = pd.concat([filtro_dormitorios_3, filtro_dormitorios_4], ignore_index=True)
imoveis_dormitorios = imoveis_dormitorios[['data', 'valor', 'tipo_transacao','estado','dormitorios']]
#comentario especial para a Gi, sobre a multiplicação por milhões utilizando '_' como separador para ficar mais fácil a visualização
imoveis_dormitorios['valor']=imoveis_dormitorios['valor']*1_000_000


In [65]:
imoveis_dormitorios

,data,valor,tipo_transacao,estado,dormitorios
0,2018-01-31,5000000.0,imoveis,rr,1
1,2018-01-31,779000000.0,imoveis,rs,1
2,2018-01-31,206000000.0,imoveis,sc,1
3,2018-01-31,14000000.0,imoveis,se,1
4,2018-01-31,550000000.0,imoveis,rj,1
...,...,...,...,...,...
8425,2024-04-30,19000000.0,imoveis,se,4
8426,2024-04-30,10000000.0,imoveis,to,4
8427,2024-04-30,499000000.0,imoveis,sp,4
8428,2024-04-30,60000000.0,imoveis,rs,4


In [66]:
filtro_tipo_imoveis = filtro_imoveis[filtro_imoveis['info'].str.contains(r'\wtipo\w', regex=True, case=False)]
filtro_tipo_imoveis['tipo'] = filtro_tipo_imoveis['info'].str.extract(r'tipo_([^_]+)')
filtro_tipo_imoveis = filtro_tipo_imoveis[['data', 'valor', 'estado', 'tipo']]
filtro_tipo_imoveis['valor'] = filtro_tipo_imoveis['valor']*1_000_000

C:\Users\diogo\AppData\Local\Temp\ipykernel_17824\906829873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_tipo_imoveis['tipo'] = filtro_tipo_imoveis['info'].str.extract(r'tipo_([^_]+)')


In [67]:
imoveis_tipo_garantia = filtro_imoveis[filtro_imoveis['info'].str.contains(r'\wgarantia\w', regex=True, case=False)]
imoveis_tipo_garantia['garantia'] = imoveis_tipo_garantia['info'].str.extract(r'garantia_([^_]+)')
imoveis_tipo_garantia = imoveis_tipo_garantia[['data', 'valor', 'estado', 'garantia']]
imoveis_tipo_garantia['valor'] = imoveis_tipo_garantia['valor']*1_000_000

C:\Users\diogo\AppData\Local\Temp\ipykernel_17824\4078544750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imoveis_tipo_garantia['garantia'] = imoveis_tipo_garantia['info'].str.extract(r'garantia_([^_]+)')


In [68]:
imoveis_area = filtro_imoveis[filtro_imoveis['info'].str.contains(r'\warea\w', regex=True, case=False)]
imoveis_area['area'] = imoveis_area['info'].str.extract(r'area_([^_]+)')
imoveis_area = imoveis_area[['data', 'valor', 'estado', 'area']]
imoveis_area.columns= ['data', 'valor m2', 'estado', 'area']

C:\Users\diogo\AppData\Local\Temp\ipykernel_17824\1162883796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imoveis_area['area'] = imoveis_area['info'].str.extract(r'area_([^_]+)')


In [69]:
imoveis_venda = filtro_imoveis[filtro_imoveis['info'].str.contains(r'\wvalor\w', regex=True, case=False)]
imoveis_venda['venda'] = imoveis_venda['info'].str.extract(r'valor_([^_]+)')
imoveis_venda= imoveis_venda[['data', 'valor', 'estado', 'venda']]

C:\Users\diogo\AppData\Local\Temp\ipykernel_17824\1265954608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imoveis_venda['venda'] = imoveis_venda['info'].str.extract(r'valor_([^_]+)')
